In [1]:
# !pip install openai langchain
# !pip install langchain_openai

!pip install pydantic

Defaulting to user installation because normal site-packages is not writeable


In [36]:
import os

#Setting Environment variable
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://dskumar.openai.azure.com/'
os.environ["AZURE_OPENAI_API_KEY"] ="62855d6dd08945819bf83aee0c104127"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] ="DskumarDeployment"
os.environ['OPENAI_TYPE']="Azure"
os.environ["LLM_MODEL"] = "gpt-35-turbo-16k"
os.environ["LLM_EMBEDDING_MODEL"] = "dskumar-text-embedding-ada-002"

from langchain_core.messages import AIMessage,SystemMessage,HumanMessage
from langchain_openai import AzureChatOpenAI

azurechatmodel = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

**JSON parser**

This output parser allows users to specify an arbitrary JSON schema and query LLMs for outputs that conform to that schema.

Keep in mind that large language models are leaky abstractions! You’ll have to use an LLM with sufficient capacity to generate well-formed JSON. In the OpenAI family, DaVinci can do reliably but Curie’s ability already drops off dramatically.

You can optionally use Pydantic to declare your data model.

In [3]:
from typing import List

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [4]:
# Define your desired data structure.

class Ouptut(BaseModel):
    excd: str = Field(description="excd")
    claimNotes: str = Field(description="claim Notes")

class Claim(BaseModel):
    claimId: str = Field(description="claimId")
    # excd: str = Field(description="excd")
    billType : int = Field (description="billType")
    reimbursementMethod : int = Field (description="reimbursement Method")
    duplicate :bool = Field(description="duplicate")
    denyReason : str = Field (description="Deny reason")
    reimbursementMethod : str = Field (description="Reimbursment method")
    hasAdditionalRevCodes : bool = Field (description="has  Additional Rev Codes ")
    pending :bool = Field(description="pending")

    # excd : str = Field(description= "output or expected excd")
    # claimNotes: str  = Field(description="claim Notes")


# Define your desired data structure.
class Claims(BaseModel):
    currentClaim : Claim = Field(description=" current claim")
    previousClaim : Claim = Field(description="previous Claim  ")
    output_current : Ouptut = Field(description="Ouptut for current claim")
    # output_previous : Ouptut = Field(description="Ouptut for previous claim")

class ClaimsArr(BaseModel):
    claimsarr : list[Claims]

In [ ]:
# class Claim(BaseModel):
#     currentClaim:  object = Field(description="question to set up a joke")

#     punchline: str = Field(description="answer to resolve the joke")

In [5]:
# And a query intented to prompt a language model to populate the data structure.
joke_query = """
You are a QA analyst, your job is to generate test data for testing Drools rules.  Please use "Drool File" to arrive at the test data.  Test data should test all possible flows in "Drool File".
Generate testdata as JSON as mentioned in the "output" only and follow the "Instructions" to generate test cases.


Instructions :

test data should cover positive flows , negative flows and edge cases as well.
Give the output only in the "output format".
Give the output as one single json
generate 10 testcase data

Drool File:
package com.edw.queries;

unit ClaimUnitData;

import com.edw.model.Claim;
import com.edw.model.Detail;

rule NonPaymentZeroPay when
$D: /details[currentClaim.billType == 0]
then
modify($D.currentClaim) {
    setExcd("5JU"),
    setClaimNotes("Claim submitted with bill type 0 provider does not expect payment.")
};
System.out.println("NonPaymentZeroPay "+$D);
end
"""


# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=ClaimsArr)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | azurechatmodel | parser

outpt = chain.invoke({"query": joke_query})



In [34]:
import json

In [7]:
print(type(outpt))

print(json.dumps(outpt))

<class 'dict'>
{"claimsarr": [{"currentClaim": {"claimId": "1", "billType": 0, "reimbursementMethod": "method1", "duplicate": true, "denyReason": "reason1", "hasAdditionalRevCodes": false, "pending": false}, "previousClaim": {"claimId": "2", "billType": 1, "reimbursementMethod": "method2", "duplicate": false, "denyReason": "reason2", "hasAdditionalRevCodes": true, "pending": true}, "output_current": {"excd": "5JU", "claimNotes": "Claim submitted with bill type 0 provider does not expect payment."}}, {"currentClaim": {"claimId": "3", "billType": 1, "reimbursementMethod": "method3", "duplicate": true, "denyReason": "reason3", "hasAdditionalRevCodes": false, "pending": true}, "previousClaim": {"claimId": "4", "billType": 2, "reimbursementMethod": "method4", "duplicate": false, "denyReason": "reason4", "hasAdditionalRevCodes": true, "pending": false}, "output_current": {"excd": "", "claimNotes": ""}}, {"currentClaim": {"claimId": "5", "billType": 2, "reimbursementMethod": "method5", "dupli

In [4]:
class info(BaseModel):
    postman_id: str = Field(description="_postman_id")
    name : str = Field (description="name")
    _schema: str = Field (description="schema")
    exporter_id : int = Field(description = "_exporter_id")    

In [5]:
class script(BaseModel):
    exec : list = Field(description="exec")
    type : str = Field(description="type")
    packages : str = Field(description="packages")

In [6]:
class event(BaseModel):
    listen: str = Field(description="listen")
    script = script

In [7]:
class raw(BaseModel):
    language : str = Field(description="language")

In [8]:
class options(BaseModel):
    raw = raw

In [9]:
class body(BaseModel):
    mode : str = Field(description="mode")
    raw : str = Field(description="raw")
    options = options        

In [10]:
class url(BaseModel):
    raw : str = Field(description="raw")
    protocol : str = Field(description="protocol")
    host : list = Field(description="host")
    port : str = Field(description="port")
    path : list = Field(description="path")

In [11]:
class request(BaseModel):
    method : str = Field(description="method")
    header : str = Field(description="header")
    body = body
    url = url
    response : list = Field(description="response")

In [12]:
class item(BaseModel):
    name : str = Field(description = "name")
    event = list[event]
    request = request
    response : str = Field(description = "response")  

TypeError: issubclass() arg 1 must be a class

In [15]:
class items(BaseModel):
    item : list[item]

NameError: name 'item' is not defined

In [13]:
from pydantic import BaseModel, Field
from typing import List, Dict

In [14]:
class script(BaseModel):
    exec : list = Field(description="exec")
    type : str = Field(description="type")
    packages : str = Field(description="packages")

In [22]:
class Event(BaseModel):
    listen: str = Field(description="listen")
    script = script

class raw(BaseModel):
    language : str = Field(description="language")

class options(BaseModel):
    raw = raw

class body(BaseModel):
    mode : str = Field(description="mode")
    raw : str = Field(description="raw")
    options = options        

class url(BaseModel):
    raw : str = Field(description="raw")
    protocol : str = Field(description="protocol")
    host : list = Field(description="host")
    port : str = Field(description="port")
    path : list = Field(description="path")

In [24]:
class Request(BaseModel):
    method : str = Field(description="method")
    header : str = Field(description="header")
    body = body
    url = url
    response : list = Field(description="response")

In [26]:
class item(BaseModel):
    name: str = Field(description="name")
    event: List[Event]
    request: Request
    response: List[str] = Field(description="response")

In [27]:
class items(BaseModel):
    item : list[item]

class info(BaseModel):
    postman_id: str = Field(description="_postman_id")
    name : str = Field (description="name")
    _schema: str = Field (description="schema")
    exporter_id : int = Field(description = "_exporter_id")    

In [28]:
from typing import List, Dict
from pydantic import BaseModel


class Script(BaseModel):
    exec: List[str]
    type: str
    packages: Dict


class Event(BaseModel):
    listen: str
    script: Script


class Body(BaseModel):
    mode: str
    raw: str
    options: Dict


class Url(BaseModel):
    raw: str
    protocol: str
    host: List[str]
    port: str
    path: List[str]


class Request(BaseModel):
    method: str
    header: List[str]
    body: Body
    url: Url


class Item(BaseModel):
    name: str
    event: List[Event]
    request: Request
    response: List


class Info(BaseModel):
    _postman_id: str
    name: str
    schema: str
    _exporter_id: str


class TestCollection(BaseModel):
    info: Info
    item: List[Item]


test_collection_data = {
    "info": {
        "_postman_id": "6259f6b2-9049-454c-95e4-b82805bfa3f8",
        "name": "testcollection",
        "schema": "https://schema.getpostman.com/json/collection/v2.1.0/collection.json",
        "_exporter_id": "98974"
    },
    "item": [
        {
            "name": "http://localhost:8080/get-claim",
            "event": [
                {
                    "listen": "prerequest",
                    "script": {
                        "exec": [
                            "// pm.collectionVariables.set(\"c_billType\",data[\"currentClaim\"][\"billType\"]);"
                        ],
                        "type": "text/javascript",
                        "packages": {}
                    }
                },
                {
                    "listen": "test",
                    "script": {
                        "exec": [
                            "pm.test(\"Status code is 200\", function () {\r",
                            "    pm.response.to.have.status(200);\r",
                            "});\r",
                            "pm.test(\"excd\", function () {\r",
                            "    var jsonData = pm.response.json();\r",
                            "    pm.expect(jsonData[0].currentClaim.excd).eq(\"5JU\")\r",
                            "});"
                        ],
                        "type": "text/javascript",
                        "packages": {}
                    }
                }
            ],
            "request": {
                "method": "POST",
                "header": [],
                "body": {
                    "mode": "raw",
                    "raw": "{\"details\":[    {  \r\n        \"currentClaim\": {\r\n            \"claimId\": \"123\",\r\n            \"billType\": 0,\r\n            \"reimbursementMethod\": \"cash\",\r\n            \"duplicate\": false,\r\n            \"denyReason\": \"\",\r\n            \"hasAdditionalRevCodes\": false,\r\n            \"pending\": true\r\n        },\r\n        \"previousClaim\": {\r\n            \"claimId\": \"456\",\r\n            \"billType\": 2,\r\n            \"reimbursementMethod\": \"check\",\r\n            \"duplicate\": true,\r\n            \"denyReason\": \"Invalid claim\",\r\n            \"hasAdditionalRevCodes\": true,\r\n            \"pending\": false\r\n        } }  \r\n      ]}",
                    "options": {
                        "raw": {
                            "language": "json"
                        }
                    }
                },
                "url": {
                    "raw": "http://localhost:8080/get-claim",
                    "protocol": "http",
                    "host": [
                        "localhost"
                    ],
                    "port": "8080",
                    "path": [
                        "get-claim"
                    ]
                }
            },
            "response": []
        },
        {
            "name": "http://localhost:8080/get-claim",
            "event": [
                {
                    "listen": "prerequest",
                    "script": {
                        "exec": [
                            "// pm.collectionVariables.set(\"c_billType\",data[\"currentClaim\"][\"billType\"]);"
                        ],
                        "type": "text/javascript",
                        "packages": {}
                    }
                },
                {
                    "listen": "test",
                    "script": {
                        "exec": [
                            "pm.test(\"Status code is 200\", function () {\r",
                            "    pm.response.to.have.status(200);\r",
                            "});\r",
                            "pm.test(\"excd\", function () {\r",
                            "    var jsonData = pm.response.json();\r",
                            "    pm.expect(jsonData[0].currentClaim.excd).eq(\"5JU\")\r",
                            "});"
                        ],
                        "type": "text/javascript",
                        "packages": {}
                    }
                }
            ],
            "request": {
                "method": "POST",
                "header": [],
                "body": {
                    "mode": "raw",
                    "raw": "{\"details\":[    {  \r\n        \"currentClaim\": {\r\n            \"claimId\": \"123\",\r\n            \"billType\": 5,\r\n            \"reimbursementMethod\": \"cash\",\r\n            \"duplicate\": false,\r\n            \"denyReason\": \"\",\r\n            \"hasAdditionalRevCodes\": false,\r\n            \"pending\": true\r\n        },\r\n        \"previousClaim\": {\r\n            \"claimId\": \"456\",\r\n            \"billType\": 2,\r\n            \"reimbursementMethod\": \"check\",\r\n            \"duplicate\": true,\r\n            \"denyReason\": \"Invalid claim\",\r\n            \"hasAdditionalRevCodes\": true,\r\n            \"pending\": false\r\n        } }  \r\n      ]}",
                    "options": {
                        "raw": {
                            "language": "json"
                        }
                    }
                },
                "url": {
                    "raw": "http://localhost:8080/get-claim",
                    "protocol": "http",
                    "host": [
                        "localhost"
                    ],
                    "port": "8080",
                    "path": [
                        "get-claim"
                    ]
                }
            },
            "response": []
        }
    ]
}

test_collection = TestCollection(**test_collection_data)

print(test_collection.json(indent=4))


NameError: Field name "schema" shadows a BaseModel attribute; use a different field name with "alias='schema'".

In [29]:
from typing import List, Dict
from pydantic import BaseModel


class Script(BaseModel):
    exec: List[str]
    type: str
    packages: Dict


class Event(BaseModel):
    listen: str
    script: Script


class Body(BaseModel):
    mode: str
    raw: str
    options: Dict


class Url(BaseModel):
    raw: str
    protocol: str
    host: List[str]
    port: str
    path: List[str]


class Request(BaseModel):
    method: str
    header: List[str]
    body: Body
    url: Url


class Item(BaseModel):
    name: str
    event: List[Event]
    request: Request
    response: List


class Info(BaseModel):
    _postman_id: str
    name: str
    schema_alias: str
    _exporter_id: str


class TestCollection(BaseModel):
    info: Info
    item: List[Item]


test_collection_data = {
    "info": {
        "_postman_id": "6259f6b2-9049-454c-95e4-b82805bfa3f8",
        "name": "testcollection",
        "schema_alias": "https://schema.getpostman.com/json/collection/v2.1.0/collection.json",
        "_exporter_id": "98974"
    },
    "item": [
        {
            "name": "http://localhost:8080/get-claim",
            "event": [
                {
                    "listen": "prerequest",
                    "script": {
                        "exec": [
                            "// pm.collectionVariables.set(\"c_billType\",data[\"currentClaim\"][\"billType\"]);"
                        ],
                        "type": "text/javascript",
                        "packages": {}
                    }
                },
                {
                    "listen": "test",
                    "script": {
                        "exec": [
                            "pm.test(\"Status code is 200\", function () {\r",
                            "    pm.response.to.have.status(200);\r",
                            "});\r",
                            "pm.test(\"excd\", function () {\r",
                            "    var jsonData = pm.response.json();\r",
                            "    pm.expect(jsonData[0].currentClaim.excd).eq(\"5JU\")\r",
                            "});"
                        ],
                        "type": "text/javascript",
                        "packages": {}
                    }
                }
            ],
            "request": {
                "method": "POST",
                "header": [],
                "body": {
                    "mode": "raw",
                    "raw": "{\"details\":[    {  \r\n        \"currentClaim\": {\r\n            \"claimId\": \"123\",\r\n            \"billType\": 0,\r\n            \"reimbursementMethod\": \"cash\",\r\n            \"duplicate\": false,\r\n            \"denyReason\": \"\",\r\n            \"hasAdditionalRevCodes\": false,\r\n            \"pending\": true\r\n        },\r\n        \"previousClaim\": {\r\n            \"claimId\": \"456\",\r\n            \"billType\": 2,\r\n            \"reimbursementMethod\": \"check\",\r\n            \"duplicate\": true,\r\n            \"denyReason\": \"Invalid claim\",\r\n            \"hasAdditionalRevCodes\": true,\r\n            \"pending\": false\r\n        } }  \r\n      ]}",
                    "options": {
                        "raw": {
                            "language": "json"
                        }
                    }
                },
                "url": {
                    "raw": "http://localhost:8080/get-claim",
                    "protocol": "http",
                    "host": [
                        "localhost"
                    ],
                    "port": "8080",
                    "path": [
                        "get-claim"
                    ]
                }
            },
            "response": []
        },
        {
            "name": "http://localhost:8080/get-claim",
            "event": [
                {
                    "listen": "prerequest",
                    "script": {
                        "exec": [
                            "// pm.collectionVariables.set(\"c_billType\",data[\"currentClaim\"][\"billType\"]);"
                        ],
                        "type": "text/javascript",
                        "packages": {}
                    }
                },
                {
                    "listen": "test",
                    "script": {
                        "exec": [
                            "pm.test(\"Status code is 200\", function () {\r",
                            "    pm.response.to.have.status(200);\r",
                            "});\r",
                            "pm.test(\"excd\", function () {\r",
                            "    var jsonData = pm.response.json();\r",
                            "    pm.expect(jsonData[0].currentClaim.excd).eq(\"5JU\")\r",
                            "});"
                        ],
                        "type": "text/javascript",
                        "packages": {}
                    }
                }
            ],
            "request": {
                "method": "POST",
                "header": [],
                "body": {
                    "mode": "raw",
                    "raw": "{\"details\":[    {  \r\n        \"currentClaim\": {\r\n            \"claimId\": \"123\",\r\n            \"billType\": 5,\r\n            \"reimbursementMethod\": \"cash\",\r\n            \"duplicate\": false,\r\n            \"denyReason\": \"\",\r\n            \"hasAdditionalRevCodes\": false,\r\n            \"pending\": true\r\n        },\r\n        \"previousClaim\": {\r\n            \"claimId\": \"456\",\r\n            \"billType\": 2,\r\n            \"reimbursementMethod\": \"check\",\r\n            \"duplicate\": true,\r\n            \"denyReason\": \"Invalid claim\",\r\n            \"hasAdditionalRevCodes\": true,\r\n            \"pending\": false\r\n        } }  \r\n      ]}",
                    "options": {
                        "raw": {
                            "language": "json"
                        }
                    }
                },
                "url": {
                    "raw": "http://localhost:8080/get-claim",
                    "protocol": "http",
                    "host": [
                        "localhost"
                    ],
                    "port": "8080",
                    "path": [
                        "get-claim"
                    ]
                }
            },
            "response": []
        }
    ]
}

test_collection = TestCollection(**test_collection_data)

print(test_collection.json(indent=4))


{
    "info": {
        "name": "testcollection",
        "schema_alias": "https://schema.getpostman.com/json/collection/v2.1.0/collection.json"
    },
    "item": [
        {
            "name": "http://localhost:8080/get-claim",
            "event": [
                {
                    "listen": "prerequest",
                    "script": {
                        "exec": [
                            "// pm.collectionVariables.set(\"c_billType\",data[\"currentClaim\"][\"billType\"]);"
                        ],
                        "type": "text/javascript",
                        "packages": {}
                    }
                },
                {
                    "listen": "test",
                    "script": {
                        "exec": [
                            "pm.test(\"Status code is 200\", function () {\r",
                            "    pm.response.to.have.status(200);\r",
                            "});\r",
                            "pm.test(\

In [48]:
from typing import List, Dict
from pydantic import BaseModel


class Script(BaseModel):
    exec: List[str]
    type: str
    packages: Dict


class Event(BaseModel):
    listen: str
    script: Script


class Body(BaseModel):
    mode: str
    raw: str
    options: Dict


class Url(BaseModel):
    raw: str
    protocol: str
    host: List[str]
    port: str
    path: List[str]


class Request(BaseModel):
    method: str
    header: List[str]
    body: Body
    url: Url


class Item(BaseModel):
    name: str
    event: List[Event]
    request: Request
    response: List


class Info(BaseModel):
    #postman_id: str
    #name: str
    #schema_alias: str
    #exporter_id: str
    _postman_id: str = Field(description="_postman_id")
    name : str = Field (description="name")
    _schema: str = Field (description="_schema")
    exporter_id : int = Field(description = "_exporter_id") 

class TestCollection(BaseModel):
    info: Info
    item: List[Item]

In [49]:
json_query = """
You are a QA analyst, your job is to generate test data for testing Drools rules.  Please use "Drool File" to arrive at the test data.  Test data should test all possible flows in "Drool File".
Generate testdata as JSON as mentioned in the "output" only and follow the "Instructions" to generate test cases.


Instructions :

test data should cover positive flows , negative flows and edge cases as well.
Give the output only in the "output format".
Give the output as one single json
generate 10 testcase data

Drool File:
package com.edw.queries;

unit ClaimUnitData;

import com.edw.model.Claim;
import com.edw.model.Detail;

rule NonPaymentZeroPay when
$D: /details[currentClaim.billType == 0]
then
modify($D.currentClaim) {
    setExcd("5JU"),
    setClaimNotes("Claim submitted with bill type 0 provider does not expect payment.")
};
System.out.println("NonPaymentZeroPay "+$D);
end
"""


# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=TestCollection)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | azurechatmodel | parser

outpt = chain.invoke({"query": json_query})

In [32]:
outpt

In [50]:
print(json.dumps(outpt))

{"info": {"name": "Test Data", "exporter_id": 12345}, "item": [{"name": "Test Case 1", "event": [{"listen": "testEvent1", "script": {"exec": ["testScript1"], "type": "scriptType1", "packages": {}}}], "request": {"method": "GET", "header": ["Header1", "Header2"], "body": {"mode": "raw", "raw": "RawData1", "options": {}}, "url": {"raw": "http://example.com", "protocol": "http", "host": ["example", "com"], "port": "80", "path": ["path1", "path2"]}}, "response": []}, {"name": "Test Case 2", "event": [{"listen": "testEvent2", "script": {"exec": ["testScript2"], "type": "scriptType2", "packages": {}}}], "request": {"method": "POST", "header": ["Header3", "Header4"], "body": {"mode": "raw", "raw": "RawData2", "options": {}}, "url": {"raw": "https://example.com", "protocol": "https", "host": ["example", "com"], "port": "443", "path": ["path3", "path4"]}}, "response": []}, {"name": "Test Case 3", "event": [{"listen": "testEvent3", "script": {"exec": ["testScript3"], "type": "scriptType3", "pack